In [16]:

import ntplib
from time import ctime


def get_ntp_offset(server):
    client = ntplib.NTPClient()
    try:
        response = client.request(server, version=3)
        print(f"Time from server: {ctime(response.tx_time)}")
        print(f"Offset: {response.offset} seconds")
        return response.offset
    except Exception as e:
        print(f"Failed to get NTP response: {e}")
        return None



In [17]:
#first, we get the offset from our locatl machine
get_ntp_offset('185.214.140.14') 


Time from server: Mon Jul  7 15:55:24 2025
Offset: -71.56516075134277 seconds


-71.56516075134277


* then, we get the data from RIPE  Atlas measurements reporting offset of +70s
* See Marco's Davids comments on the Atlas mailing list: https://mailman.ripe.net/archives/list/ripe-atlas@ripe.net/thread/GEYE6ZH2BECS7VEDEUO5I7JM4DIF3UHK/

In [18]:
import requests

#let's get measurements reported by Marco Davids on 

url = "https://atlas.ripe.net/api/v2/measurements/115803954/results/?format=json"

try:
    response = requests.get(url)
    response.raise_for_status()  # Raise exception for HTTP errors
    data = response.json()       # Parse JSON into a Python dict
    print(f"Downloaded {len(data)} records.")
except requests.RequestException as e:
    print(f"Error downloading data: {e}")

# 'data' now contains the JSON response as a Python dictionary/list


Downloaded 3 records.


In [34]:
'''
offsets are defined in RFC5905:

 The four most recent timestamps, T1 through T4, are used to compute
   the offset of B relative to A

   theta = T(B) - T(A) = 1/2 * [(T2-T1) + (T3-T4)]
   
   
   In the case of a stateless server, the protocol can be
   simplified.  A stateless server copies T3 and T4 from the client
   packet to T1 and T2 of the server packet and tacks on the transmit
   timestamp T3 before sending it to the client. 
      
   
   
'''
def calc_offset(res_dict):
    
    offset=(((res_dict['receive-ts'] - res_dict['origin-ts'])
             + (res_dict['transmit-ts'] - res_dict['final-ts'])) / 2)
    return offset


In [35]:
#let's compute the offset from the measurement data

print('probe_id,offset')
for k in data:
    for x in k['result']:
        
        offset = calc_offset(x)

        print(k['prb_id'], offset)

probe_id,offset
1005259 -71.55228471755981
1005259 -71.55228662490845
1005259 -71.55228686332703
1009428 -71.55045318603516
1009428 -71.55063533782959
1009428 -71.55987024307251
6567 -71.55196952819824
6567 -71.55212187767029
6567 -71.55217170715332


In [41]:
# It appears that Atlas is reversing the order of the algorithms.
# This behavior does not align with the specification defined in the RFC.

#let's write a function which inverts the order of the variables to confirm it

def calc_offset_inverted(res_dict):
    
    offset=(((res_dict['origin-ts'] - res_dict['receive-ts']  )
             + (res_dict['final-ts'] - res_dict['transmit-ts']  )) / 2)
    return offset

In [42]:
#let's compute the inverted offset from the measurement data

print('probe_id,offset')
for k in data:
    for x in k['result']:
        
        offset = calc_offset_inverted(x)

        print(k['prb_id'], offset)

probe_id,offset
1005259 71.55228471755981
1005259 71.55228662490845
1005259 71.55228686332703
1009428 71.55045318603516
1009428 71.55063533782959
1009428 71.55987024307251
6567 71.55196952819824
6567 71.55212187767029
6567 71.55217170715332
